In [2]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [37]:
import numpy as np

In [13]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

## Step1: Localizing & Extracting Faces

In [3]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [6]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

In [7]:
#extract a single face from a given photograph
def extract_face(filename, required_size = (160, 160)):
    #load image from file
    image = Image.open(filename)
    #convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

#load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    filenames_list = list() #NJ: A list to store all the images file names
    
    #enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        # store
        faces.append(face)
        filenames_list.append(path) 
        
    return faces, filenames_list

#load & extract faces for a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y, path_list = list(), list(), list()
    
    # enumerate folders, one per class
    for subdir in listdir(directory):
        #path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces, filenames_list = load_faces(path)
        
        # create labels
        labels = [subdir for _ in range(len(faces))]
        
        # summarize progress
        print ('>loaded %d example for class %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
        path_list.extend(filenames_list) # to get the actual image names
        
    return asarray(X), asarray(y), asarray(path_list)

In [8]:
def load_dataset(directory):
    X, y, path_list = list(), list(), list()
    
    # enumerate folders, one per class
    for subdir in listdir(directory):
        #path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces, filenames_list = load_faces(path)
        
        # create labels
        labels = [subdir for _ in range(len(faces))]
        
        # summarize progress
        print ('>loaded %d example for class %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
        path_list.extend(filenames_list) # to get the actual image names
        
    return asarray(X), asarray(y), asarray(path_list)

## Faces Extraction from dataset

In [9]:
# load train dataset
dataset_train_path = './dataset/train/'

faces_dataset = load_dataset(dataset_train_path)

>loaded 14 example for class ben_afflek
>loaded 16 example for class elton_john
>loaded 21 example for class jerry_seinfeld
>loaded 19 example for class madonna
>loaded 21 example for class mindy_kaling


In [10]:
savez_compressed('./faces-train_dataset.npz', faces_dataset[0], faces_dataset[1], faces_dataset[2])

In [14]:
data = load('./faces-train_dataset.npz')

In [15]:
# load the face dataset
data = load('./faces-train_dataset.npz')

trainX, trainy, train_images_path = data['arr_0'], data['arr_1'], data['arr_2'] #, data['arr_3']
print('Loaded Train dataset: ', trainX.shape, trainy.shape, train_images_path.shape)

Loaded Train dataset:  (91, 160, 160, 3) (91,) (91,)


## Feature Extraction for the Faces extracted from Dataset

In [16]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet

embedder = FaceNet()

In [17]:
features_dataset = embedder.embeddings(trainX)

In [18]:
# save arrays to one file in compressed format
savez_compressed('./faces-train_embeddings.npz', features_dataset, trainy, train_images_path)

In [19]:
from numpy import load

#load dataset
data = load('./faces-train_embeddings.npz')
trainX_features, trainy, train_images_path = data['arr_0'], data['arr_1'], data['arr_2'] #, data['arr_3']

In [114]:
def get_duplicate_images_df(feature, matching_cutoff_threshold = 0.65):
    img_dup = pd.DataFrame([],columns=['image_id','Image_label','Image_path','Duplicate_image','Duplicate_path'])
    for id1,feature1 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
        for id2,feature2 in enumerate(zip(feature['arr_0'], feature['arr_1'], feature['arr_2'])):
            if id2>id1:
                image_1,label_1,path_1 = feature1[0],feature1[1],feature1[2]
                image_2,label_2,path_2 = feature2[0],feature2[1],feature2[2]

                similarity_score = cosine_similarity(image_1.reshape(1, -1),
                                                     image_2.reshape(1, -1))
                if similarity_score > matching_cutoff_threshold:
                    img_dup.loc[len(img_dup)]= [id1,label_1,path_1,label_2,path_2]
    print('x:',x)
    return img_dup

In [115]:
df1 = get_duplicate_images_df(data)

x: 91


In [116]:
# cosine_similarity(trainX_features[0].reshape(1, -1),trainX_features[90].reshape(1, -1))

array([[-0.07182173]], dtype=float32)

In [111]:
df1['image_id']

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
32       0
33       0
34       0
35       0
36       0
37       0
38       0
39       0
40       0
41       0
42       0
43       0
44       0
45       0
46       0
47       0
48       0
49       0
50       0
51       0
52       0
53       0
54       0
55       0
56       0
57       1
58       1
59       1
60       1
61       1
62       1
63       1
64       1
65       1
66       1
67       1
68       1
69       1
70       1
71       1
72       1
73       1
74       1
75       1
76       1
77       1
78       1
79       1
80       1
81       1
82       1
83       1
84       1
85       1
86       1
87       1
88       1
89       1
90       1

In [110]:
pd.set_option('display.max_rows', 2200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [99]:
df1['image_id'].value_counts()

30    18
70    14
32    14
35    13
33    13
36    12
31    12
73    12
39    10
77    10
40    10
38    10
34    10
79     9
51     8
80     8
81     7
78     6
44     6
43     6
41     6
82     5
83     5
45     5
42     4
46     4
19     4
37     4
2      4
6      4
47     3
57     3
86     3
1      3
16     3
22     3
74     3
72     3
85     3
71     2
64     2
62     2
5      2
17     2
18     2
48     2
4      2
8      2
20     2
10     2
3      2
23     2
87     2
75     1
0      1
66     1
65     1
54     1
53     1
49     1
25     1
24     1
21     1
12     1
11     1
9      1
88     1
Name: image_id, dtype: int64

In [89]:
img_dup

,Image_label,Image_path,Duplicate_image,Duplicate_path
0,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (1...,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (2...
1,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (1...,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (2...
2,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (1...,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (6...
3,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (1...,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (9...
4,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (1...,ben_afflek,./dataset/train/ben_afflek/train_ben_afflek (1...
...,...,...,...,...
317,mindy_kaling,./dataset/train/mindy_kaling/train_mindy_kalin...,mindy_kaling,./dataset/train/mindy_kaling/train_mindy_kalin...
318,mindy_kaling,./dataset/train/mindy_kaling/train_mindy_kalin...,mindy_kaling,./dataset/train/mindy_kaling/train_mindy_kalin...
319,mindy_kaling,./dataset/train/mindy_kaling/train_mindy_kalin...,mindy_kaling,./dataset/train/mindy_kaling/train_mindy_kalin...
320,mindy_kaling,./dataset/train/mindy_kaling/train_mindy_kalin...,mindy_kaling,./dataset/train/mindy_kaling/train_mindy_kalin...


In [83]:
trainX_features.shape

(91, 512)

In [84]:
91*91

8281

In [47]:
matching_cutoff_threshold = 0.65
match_found_indices = np.argwhere(similarity_score > matching_cutoff_threshold)

In [49]:
match_found_indices[0]

array([0, 0], dtype=int64)

In [51]:
match_found_indices[734]

array([90, 90], dtype=int64)

In [85]:
x = [5,6,7,8]
y = ['A','B','C','D']

In [87]:
for idx,val in enumerate(zip(x,y)):
    print(idx,'->',val[0],'->',val[1])

0 -> 5 -> A
1 -> 6 -> B
2 -> 7 -> C
3 -> 8 -> D


In [55]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def get_duplicate_images(test_images_folder, matching_cutoff_threshold = 0.65): #Lower Threshold can give more False Positive while higher threshold can miss some of the matches
    
    #load the test dataset - Extracting the Faces
    testX, testy, test_images_path = load_dataset(dataset_test_path)
    
    #Getting the feature vectors - Feature Extraction
    embedder = FaceNet()
    # Feature Extraction
    testX_features = embedder.embeddings(testX)
    print(testX_features.shape, testy.shape, test_images_path.shape)

    # Getting the Cosine Similarity
    similarity_score = cosine_similarity(testX_features, trainX_features)
    # temp = cosine_similarity(testX[0].reshape(1, -1), trainX[0:10])
    print('similarity_score.shape', similarity_score.shape)
    
    match_found_indices = np.argwhere(similarity_score > matching_cutoff_threshold) #np.where
    match_found_indices.shape


    matched_images_list = []

    for i in range(match_found_indices.shape[0]):

        test_image_id = match_found_indices[i][0]
        train_image_id = match_found_indices[i][1]

        test_class_name = testy[test_image_id]
        test_image_path = test_images_path [test_image_id]

        matched_class_name = trainy[train_image_id]
        matched_image_path = train_images_path[train_image_id]

        match = (test_image_id, test_class_name, test_image_path, matched_class_name, matched_image_path)
        matched_images_list.append(match)

        #print(test_image_id, test_class_name, test_image_path, train_image_id, train_class_name)
        #print('Test Image: ', test_class_name, test_image_path, ' Matched Image:', train_class_name, train_image_path)
        #break
    
    # Converting the list to a dataframe for easy handling
    df = pd.DataFrame(matched_images_list, columns =['Test Image Id', 'Test Class', 'Test Image', 'Matched Class', 'Matched Image'])
    
    return df

In [ ]:
duplicate_images_df = get_duplicate_images(dataset_test_path, matching_cutoff_threshold)